In [1]:
!pip install BeautifulSoup4

You should consider upgrading via the 'd:\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install geocoder

You should consider upgrading via the 'd:\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup

# Part 1 - Extracting data and creating the dataframe

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = urllib.request.Request(url, headers={ 'User-Agent': 'Mozilla/5.0' })
req = urllib.request.urlopen(req)
tree = BeautifulSoup(req, 'html5lib')

In [5]:
table = tree.find_all('tr')

In [6]:
codes = []
borough = []
neigh = []

In [7]:
for x in table:
    for y in x.find_all('td'):
        text =y.get_text().strip()
        if 'Not assigned' in text:
            pass
        else:
            words = text[3:]
            texts = (words.strip(')').split('('))
            try:
                text1 = texts[0]
                text2 = texts[1]
                pcode = text[:3]
                if text2 == 'Not assigned':
                    text2 = text1
                    codes.append(pcode)
                    borough.append(text1)
                    neigh.append(text2)
                else:
                    codes.append(pcode)
                    borough.append(text1)
                    neigh.append(text2)
            except:
                pass

In [8]:
table = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])

In [9]:
table.head()

,PostalCode,Borough,Neighborhood


In [10]:
table['PostalCode'] = codes
table['Borough'] = borough
table['Neighborhood'] = neigh

In [11]:
table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


# PART 2 - Adding latitude and longitude

In [12]:
geo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [13]:
lat1 = []
long1 =[]
codes2 = []
bor=[]
neighb=[]


for x,y,z in zip(geo['Postal Code'],geo['Latitude'],geo['Longitude']):
    for n,m,o in zip(table['PostalCode'],table['Borough'],table['Neighborhood']):
        if x == n:
            lat = y
            long = z
            lat1.append(lat)
            long1.append(long)
            codes2.append(n)
            bor.append(m)
            neighb.append(o.replace(')',' '))
        else:
            pass
        

In [14]:
table2 = pd.DataFrame()
table2['Code'] = codes2
table2['Borough'] = bor
table2['Neighborhood'] = neighb
table2['Latitude'] = lat1
table2['Longitude'] = long1

In [15]:
table2.head(10)

,Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


# Part 3 - Clustering

In [16]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [17]:
address = 'Toronto City, TOR'

geolocator = Nominatim(user_agent="ny_explorer")
location1 = geolocator.geocode(address)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Toronto City are 43.7370584, -79.2442535.


In [18]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude1, longitude1], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(table2['Latitude'], table2['Longitude'], table2['Borough'], table2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto